# MSSQL Integration Test

This notebook tests the MSSQL integration to ensure everything is working correctly.

In [2]:
%load_ext autoreload
%autoreload 2
# Test 1: Import the sqlserver module
from helpers import sqlserver
print("✓ Successfully imported sqlserver module")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✓ Successfully imported sqlserver module


In [3]:
# Test 2: Get connection configuration
config = sqlserver.get_connection_config('TEST')
print("Connection Configuration:")
print(f"  Server: {config['server']}")
print(f"  Database: {config['database']}")
print(f"  User: {config['user']}")
print(f"  Driver: {config['driver']}")
print("✓ Configuration loaded successfully")

Connection Configuration:
  Server: sqlserver
  Database: test_db
  User: sa
  Driver: ODBC Driver 18 for SQL Server
✓ Configuration loaded successfully


In [4]:
# Test 3: Initialize test database (if needed)
print("Checking if test database exists...")

# Try to connect to test_db
if not sqlserver.test_connection('TEST'):
    print("Database doesn't exist or not accessible. Initializing...")
    
    # Initialize the database using the SQL script
    success = sqlserver.initialize_database('init_sqlserver.sql', connection='TEST')
    
    if success:
        print("✓ Test database initialized successfully")
    else:
        print("✗ Failed to initialize database")
        raise Exception("Database initialization failed")
else:
    print("✓ Test database already exists and is accessible")

# Verify connection works
if sqlserver.test_connection('TEST'):
    print("✓ Successfully connected to SQL Server!")
else:
    print("✗ Failed to connect to SQL Server")
    raise Exception("Cannot connect to SQL Server")

Checking if test database exists...
✓ Test database already exists and is accessible
✓ Successfully connected to SQL Server!


In [5]:
# Test 4: Verify database schema
tables = sqlserver.execute_query(
    "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE' ORDER BY TABLE_NAME",
    connection='TEST'
)

print("Database tables:")
for table in tables['TABLE_NAME']:
    print(f"  - {table}")

if 'test_portfolios' in tables['TABLE_NAME'].values and 'test_risks' in tables['TABLE_NAME'].values:
    print("\n✓ Test database schema is correctly initialized")
else:
    print("\n✗ Missing required tables")

Database tables:
  - test_portfolios
  - test_risks

✓ Test database schema is correctly initialized


/home/jovyan/workspace/helpers/sqlserver.py:459: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


In [6]:
# Test 5: Execute simple query
df = sqlserver.execute_query(
    "SELECT * FROM test_portfolios",
    connection='TEST'
)

print(f"✓ Query executed successfully! Found {len(df)} portfolios")
print("\nFirst few rows:")
display(df.head())

✓ Query executed successfully! Found 5 portfolios

First few rows:


,id,portfolio_name,portfolio_value,status,created_ts,updated_ts
0,1,Test Portfolio A,750000.0,ACTIVE,2025-11-04 22:05:37.336666,2025-11-04 22:05:37.336666
1,2,Test Portfolio B,1250000.0,ACTIVE,2025-11-04 22:05:37.340000,2025-11-04 22:05:37.340000
2,3,Test Portfolio C,500000.0,ACTIVE,2025-11-04 22:05:37.346666,2025-11-04 22:05:37.346666
3,4,Test Portfolio D,2000000.0,ACTIVE,2025-11-04 22:05:37.350000,2025-11-04 22:05:37.350000
4,5,Test Portfolio E,300000.0,INACTIVE,2025-11-04 22:05:37.356666,2025-11-04 22:05:37.356666


In [7]:
# Test 6: Query with named parameters
df = sqlserver.execute_query(
    "SELECT * FROM test_portfolios WHERE portfolio_value > {min_value}",
    params={'min_value': 1000000},
    connection='TEST'
)

print(f"✓ Query with parameters executed successfully! Found {len(df)} portfolios with value > $1M")
display(df)

✓ Query with parameters executed successfully! Found 2 portfolios with value > $1M


/home/jovyan/workspace/helpers/sqlserver.py:459: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,id,portfolio_name,portfolio_value,status,created_ts,updated_ts
0,2,Test Portfolio B,1250000.0,ACTIVE,2025-11-04 22:05:37.340,2025-11-04 22:05:37.340
1,4,Test Portfolio D,2000000.0,ACTIVE,2025-11-04 22:05:37.350,2025-11-04 22:05:37.350


In [8]:
# Test 7: Execute query from SQL file
df = sqlserver.execute_query_from_file(
    'examples/sample_query.sql',
    params={'portfolio_id': 1, 'risk_type': 'VaR_95'},
    connection='TEST'
)

print(f"✓ SQL file executed successfully! Found {len(df)} results")
display(df)

✓ SQL file executed successfully! Found 1 results


/home/jovyan/workspace/helpers/sqlserver.py:459: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,portfolio_name,portfolio_value,created_ts,risk_type,risk_value,calculated_ts
0,Test Portfolio A,750000.0,2025-11-04 22:05:37.336666,VaR_95,45000.0,2025-11-04 22:05:37.360


In [9]:
# Test 8: Execute scalar query
count = sqlserver.execute_scalar(
    "SELECT COUNT(*) FROM test_portfolios",
    connection='TEST'
)

print(f"✓ Scalar query executed successfully! Total portfolios: {count}")

✓ Scalar query executed successfully! Total portfolios: 5


## Summary

If all cells above executed successfully, the MSSQL integration is working correctly! ✅

You can now:
1. Create SQL scripts in `workspace/sql/`
2. Execute them from notebooks using `execute_query_from_file()` or `execute_script_file()`
3. Use named parameters for dynamic queries
4. Connect to production MSSQL databases by configuring connections in `.env`